In [32]:
import pandas as pd 
import json

file_path = '/home/mantovani/Documents/IC/HMC_TE-s/Temp_Data/Dados_Dominios_Conservados/sequences.json'

with open(file_path, mode='r') as file:


    dados = json.load(file)

    for i in dados['results']:

        [i.pop(j, None) for j in ['crossReferences', 'md5', 'sequence']]

        for j in i['openReadingFrames']:

            [j.pop(k, None) for k in ['id']]

            P_sequence = [j[['start', 'end', 'strand']], j['protein']['sequence']]

            print(P_sequence)

    file.close()

{'start': 162, 'end': 254, 'strand': 'ANTISENSE', 'protein': {'sequence': 'LTFIERLLCARHCSKRFTCINSFNPHNNPMR', 'md5': 'a50fc3281d42937df9cbc423228f1e0d', 'matches': [], 'xref': [{'name': 'orf7 source=DF000000001 coords=254..162 length=31 frame=6 desc=<unknown description>', 'id': 'orf7'}]}, 'nucleotideSequence': 10}
{'start': 103, 'end': 183, 'strand': 'ANTISENSE', 'protein': {'sequence': 'SSQQPYEVGTIIIPILQMRKLRHREVK', 'md5': '5bc13d86ff7c4a2974f44ad966ec795b', 'matches': [], 'xref': [{'name': 'orf8 source=DF000000001 coords=183..103 length=27 frame=5 desc=<unknown description>', 'id': 'orf8'}]}, 'nucleotideSequence': 10}
{'start': 1, 'end': 99, 'strand': 'ANTISENSE', 'protein': {'sequence': 'LAQGHTASKWQSRDSNPGSLAPEPVLLTTMLYC', 'md5': 'afa245f982e409b93f9c8103a3d18257', 'matches': [], 'xref': [{'name': 'orf11 source=DF000000001 coords=99..1 length=33 frame=5 desc=<unknown description>', 'id': 'orf11'}]}, 'nucleotideSequence': 10}
{'start': 21, 'end': 119, 'strand': 'ANTISENSE', 'protein'

In [26]:
with open(file_path, 'r') as file:
    data = json.load(file)

# Inicializando uma lista para armazenar os dados

genes = []
for gene in data:
    gene_info = {"id": gene["id"]}
    for i, domain in enumerate(gene["domains"], 1):  # Numerando os domínios: 1, 2, 3, ...
        gene_info[f"domain_{i}_name"] = domain["name"]
        gene_info[f"domain_{i}_start"] = domain["start"]
        gene_info[f"domain_{i}_end"] = domain["end"]
        gene_info[f"domain_{i}_strand"] = domain["strand"]
        gene_info[f"domain_{i}_protein_sequence"] = domain["protein_sequence"]
    genes.append(gene_info)

# Criando o DataFrame final
df = pd.DataFrame(genes)
# Exibindo o DataFrame
df.head()

TypeError: string indices must be integers, not 'str'